In [1]:
import os
import time
import config
from datetime import datetime
from log_utils import read_log_messages, normalize_template, normalize_template_v1, normalize_template_v2, save_templates
from ollama_utils import start_ollama_server, stop_ollama_server, start_ollama_server_log
from autogen import AssistantAgent
from codecarbon import EmissionsTracker
from evaluation import evaluate_and_save

In [2]:
# --- Configuration ---
llm_config = config.LLM_CONFIG
task = config.TASK_PROMPT
sys_prompt_few_shot_single_log_parser = config.SYS_MSG_SINGLE_LOG_PARSER_FEW_SHOT
sys_prompt_zero_shot_single_log_parser = config.SYS_MSG_SINGLE_LOG_PARSER_ZERO_SHOT

LOG_DIR = config.LOG_DIR
RESULT_DIR = config.RESULT_DIR
os.makedirs(RESULT_DIR, exist_ok=True)
 
# change this for different designs, i.e., "SA-zero"
DESIGN = "SA-few"
model = llm_config["config_list"][0]["model"].replace(":", "-")
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
project_name = DESIGN.capitalize()
exp_name = f"{project_name}_{model}_{timestamp}"

input_log_file = "HDFS_200_sampled.log"
log_path = os.path.join(LOG_DIR, input_log_file)
ground_truth_file_path = os.path.join(LOG_DIR, "HDFS_200_sampled_log_structured.csv")

In [ ]:
# --- Agent Creation ---
def create_single_log_parser_agent(llm_config, sys_prompt):
    return AssistantAgent(
        name="log_parser_agent",
        system_message=sys_prompt,
        description="Analyze the log message in order to determine the corresponding template.",
        llm_config=llm_config,
        human_input_mode="NEVER",
    )

In [ ]:
# --- Log Reading ---
logs = read_log_messages(log_path)

In [ ]:
# --- With CodeCarbon Emissions Tracking ---
def run_inference_with_emissions(logs, llm_config, sys_prompt_log_parser, task, exp_name, result_dir):
    parsed_templates = []
    tracker = EmissionsTracker(project_name=exp_name, output_dir=result_dir, save_to_file=True)
    tracker.start()
    try:
        log_parser = create_single_log_parser_agent(llm_config, sys_prompt_log_parser)
        for i, log_message in enumerate(logs):
            content = task + log_message
            res = log_parser.generate_reply(messages=[{"content": content, "role": "user"}])
            if res is not None and "content" in res:
                parsed_templates.append(res["content"].strip())
            else:
                parsed_templates.append("NONE")
                print(f"[Warning] Skipped log {i} — no response or invalid format.")
    finally:
        emissions = tracker.stop()
    print(f"Emissions: {emissions} kg CO2")
    return parsed_templates


In [ ]:
# --- Main Execution ---
proc = start_ollama_server()
time.sleep(5)  # Give it some time to initialize
parsed_templates = run_inference_with_emissions(logs, llm_config, sys_prompt_few_shot_single_log_parser, task, exp_name, RESULT_DIR)
save_templates(parsed_templates, llm_config, DESIGN, RESULT_DIR)
stop_ollama_server(proc)

In [ ]:
# --- Print Results ---
#print("Templates:", parsed_templates)

In [ ]:
results = evaluate_and_save(normalize_template, parsed_templates, ground_truth_file_path, exp_name)
results_v1 = evaluate_and_save(normalize_template_v1, parsed_templates, ground_truth_file_path, exp_name)
results_v2 = evaluate_and_save(normalize_template_v2, parsed_templates, ground_truth_file_path, exp_name)

In [ ]:
#print("Results:", results)